This notebook is developed using the `Python 3 (TensorFlow 2.3 Python 3.7 CPU Optimized)` kernel on an `ml.t3.medium` instance.

In [ ]:
!pip install -q sagemaker-experiments

In [ ]:
import sagemaker
import json
import boto3

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'sagemaker-studio-book/chapter05'

In [ ]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import sequence
from tensorflow.python.keras.datasets import imdb

In [ ]:
max_features = 20000
maxlen = 400

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

In [ ]:
data_dir = os.path.join(os.getcwd(), 'imdb_data')
os.makedirs(data_dir, exist_ok=True)

train_dir = os.path.join(os.getcwd(), 'imdb_data/train')
os.makedirs(train_dir, exist_ok=True)

test_dir = os.path.join(os.getcwd(), 'imdb_data/test')
os.makedirs(test_dir, exist_ok=True)

csv_test_dir = os.path.join(os.getcwd(), 'imdb_data/csv-test')
os.makedirs(csv_test_dir, exist_ok=True)

np.save(os.path.join(train_dir, 'x_train.npy'), x_train)
np.save(os.path.join(train_dir, 'y_train.npy'), y_train)
np.save(os.path.join(test_dir, 'x_test.npy'), x_test)
np.save(os.path.join(test_dir, 'y_test.npy'), y_test)
np.savetxt(os.path.join(csv_test_dir, 'csv-test.csv'), 
           np.array(x_test[:100], dtype=np.int32), fmt='%d', delimiter=",")

In [ ]:
traindata_s3_prefix = f'{prefix}/imdb_data/train'
testdata_s3_prefix = f'{prefix}/imdb_data/test'

train_s3 = sess.upload_data(path='./imdb_data/train/', key_prefix=traindata_s3_prefix)
test_s3 = sess.upload_data(path='./imdb_data/test/', key_prefix=testdata_s3_prefix)

In [ ]:
!mkdir code

In [ ]:
%%writefile code/tensorflow_sentiment.py
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
import argparse
import codecs
import json
import numpy as np
import os
import tensorflow as tf

max_features = 20000
maxlen = 400
embedding_dims = 300
filters = 256
kernel_size = 3
hidden_dims = 256

def parse_args():
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script
    parser.add_argument('--epochs', type=int, default=1)
    parser.add_argument('--batch_size', type=int, default=64)
    parser.add_argument('--learning_rate', type=float, default=0.01)
    parser.add_argument('--drop_out_rate', type=float, default=0.2)

    # data directories
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))

    # model directory /opt/ml/model default set by SageMaker
    parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR'))

    return parser.parse_known_args()


def save_history(path, history):
    history_for_json = {}
    # transform float values that aren't json-serializable
    for key in list(history.history.keys()):
        if type(history.history[key]) == np.ndarray:
            history_for_json[key] == history.history[key].tolist()
        elif type(history.history[key]) == list:
           if  type(history.history[key][0]) == np.float32 or type(history.history[key][0]) == np.float64:
               history_for_json[key] = list(map(float, history.history[key]))

    with codecs.open(path, 'w', encoding='utf-8') as f:
        json.dump(history_for_json, f, separators=(',', ':'), sort_keys=True, indent=4) 


def get_train_data(train_dir):
    x_train = np.load(os.path.join(train_dir, 'x_train.npy'))
    y_train = np.load(os.path.join(train_dir, 'y_train.npy'))
    print(f'x train {x_train.shape} y train {y_train.shape}')

    return x_train, y_train


def get_test_data(test_dir):
    x_test = np.load(os.path.join(test_dir, 'x_test.npy'))
    y_test = np.load(os.path.join(test_dir, 'y_test.npy'))
    print(f'x test {x_test.shape} y test {y_test.shape}')

    return x_test, y_test


def get_model(args):
    embedding_layer = tf.keras.layers.Embedding(max_features,
                                                embedding_dims,
                                                input_length=maxlen)

    sequence_input = tf.keras.Input(shape=(maxlen,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    x = tf.keras.layers.Dropout(args.drop_out_rate)(embedded_sequences)
    x = tf.keras.layers.Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1)(x)
    x = tf.keras.layers.MaxPooling1D()(x)
    x = tf.keras.layers.GlobalMaxPooling1D()(x)
    x = tf.keras.layers.Dense(hidden_dims, activation='relu')(x)
    x = tf.keras.layers.Dropout(args.drop_out_rate)(x)
    preds = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(sequence_input, preds)
    optimizer = tf.keras.optimizers.Adam(args.learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


if __name__ == "__main__":

    args, _ = parse_args()

    x_train, y_train = get_train_data(args.train)
    x_test, y_test = get_test_data(args.test)

    model = get_model(args)

    history = model.fit(x_train, y_train,
              batch_size=args.batch_size,
              epochs=args.epochs,
              validation_data=(x_test, y_test))

    save_history(args.model_dir + "/history.p", history)
    
    # create a TensorFlow SavedModel for deployment to a SageMaker endpoint with TensorFlow Serving
    model.save(args.model_dir + '/1')

In [ ]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from botocore.exceptions import ClientError
from time import gmtime, strftime
import time

experiment_name = 'imdb-sentiment-analysis'

try:
    experiment = Experiment.create(
        experiment_name=experiment_name, 
        description='Training a sentiment classification model using imdb dataset.')
except ClientError as e:
    print(f'{experiment_name} experiment already exists! Reusing the existing experiment.')
    

In [ ]:
from sagemaker.tensorflow import TensorFlow

exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
jobname = f'imdb-tf-{exp_datetime}'

s3_output_location = f's3://{bucket}/{prefix}/{jobname}'
code_dir = f's3://{bucket}/{prefix}/{jobname}'

train_instance_type = 'ml.p3.2xlarge'
hyperparameters = {'epochs': 10, 'batch_size': 64, 
                   'learning_rate': 0.01, 'drop_out_rate': 0.2}
model_local_path = '/opt/ml/model'

estimator = TensorFlow(source_dir='code',
                       entry_point='tensorflow_sentiment.py',
                       output_path=s3_output_location,
                       code_location=code_dir,
                       model_dir=model_local_path,
                       instance_type=train_instance_type,
                       instance_count=1,
                       enable_sagemaker_metrics=True,
                       hyperparameters=hyperparameters,
                       role=role,
                       framework_version='2.1',
                       py_version='py3')

data_channels = {'train':train_s3, 'test': test_s3}
print(data_channels)

In [ ]:
# Creating a new trial for the experiment
exp_trial = Trial.create(experiment_name=experiment_name, 
                         trial_name=jobname)

experiment_config={'ExperimentName': experiment_name,
                   'TrialName': exp_trial.trial_name,
                   'TrialComponentDisplayName': 'Training'}

estimator.fit(inputs=data_channels,
              job_name=jobname,
              experiment_config=experiment_config,
              logs=True)

In [ ]:
!mkdir ./imdb_data/model -p
!aws s3 cp {estimator.model_data} ./imdb_data/model.tar.gz
!tar -xzf ./imdb_data/model.tar.gz -C ./imdb_data/model/

In [ ]:
my_model=tf.keras.models.load_model('./imdb_data/model/1/')

In [ ]:
my_model.summary()

In [ ]:
loss, acc=my_model.evaluate(x_test, y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))